# Producer - Consumer

# Setup

In [9]:
import threading
import time
import random

from threading import Thread
from threading import Event
from threading import Lock
from threading import Condition

from queue import Queue

from concurrent.futures import ThreadPoolExecutor

# Logging

In [10]:
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logger = logging.getLogger()
#logger.setLevel(logging.DEBUG)
logger.debug('Hello')

DEBUG:root:Hello


# Event
* To kick off this notebook, we have some worker threads
* We'd like to have them pause and receive an event that has them work
* We can use the Event class to wait and signal

In [11]:
event = Event()

def wait_for_event(thread_name):
    logger.debug(f'  Waiter {thread_name}: Waiting for Event')
    event.wait()
    logger.debug(f'  Waiter {thread_name}: Received Event')
    logger.debug(f'  Waiter {thread_name}: Working')
    
logger.debug('Main Starting Waiter Threads')
waiter1 = Thread(target=wait_for_event, args=(1,))
waiter2 = Thread(target=wait_for_event, args=(2,))
waiter1.start()
waiter2.start()

logger.debug('Main Sleeping')
time.sleep(2)

logger.debug('Main Setting Event')
event.set()
event.clear()
waiter1.join()
waiter2.join()

DEBUG:root:Main Starting Waiter Threads
DEBUG:root:  Waiter 1: Waiting for Event
DEBUG:root:  Waiter 2: Waiting for Event
DEBUG:root:Main Sleeping
DEBUG:root:Main Setting Event
DEBUG:root:  Waiter 2: Received Event
DEBUG:root:  Waiter 1: Received Event
DEBUG:root:  Waiter 2: Working
DEBUG:root:  Waiter 1: Working


# Condition
* Building on the Event
* We have an amount of work to complete
* We want to use multiple workers to finish the job
* We need a mechanism for controlling access to the work list
* We need a way to signal that there is work for workers to do

In [12]:
work    = []
results = []

condition = Condition()

class Producer():
    def __init__(self, workers=5):
        self.workers = workers
        self.threads = None
        
    def create_workers(self):
        # Create daemon five worker threads
        logger.debug('Main Starting Threads')
        self.threads = [Worker(i) for i in range(self.workers)]
        for t in self.threads: t.start()
        
    def create_work(self, batches = 5, items = 5, sleeping = 1):
        logger.debug('Main Creating Work')
        for b in range(batches):
            with condition:
                for i in range(items):
                    number = b * items + i
                    logger.debug(f'Main Producing Work : {number}')
                    work.append(number)
                    condition.notifyAll()
            time.sleep(sleeping)
            
    def wait_for_shift(self):
        logger.debug('Main Waiting for Shift End')
        time.sleep(5)
            
    def end_work(self):
        logger.debug('Ending Work')
        with condition:
            [t.end() for t in threads]
            condition.notifyAll()
        logger.debug('Waiting for Workers Threads')
        for t in threads: t.join()
    
class Worker(Thread):
    def __init__(self, name):
        self.running = True
        threading.Thread.__init__(self)
    
    def end(self):
        self.running = False
        
    def run(self):
        while self.running:
            with condition:
                # No work, wait
                if len(work) == 0:
                    condition.wait()

                # Take a number off the queue & multiply by 4
                try:
                    number = work.pop()
                    logger.debug(f'  Thread {threading.current_thread().name} : Getting Value {number}')
                    time.sleep(.1)
                    result = (number, number * 4)
                    results.append(result)
                except: # Popped an Empty List -- Are we done?
                    pass
        logger.debug(f'  Thread {threading.current_thread().name} Finished')

# Run the produce function
p = Producer()
p.create_workers()
p.create_work()
p.wait_for_shift()
p.end_work()

# Print out Results
logger.debug(results)

DEBUG:root:Main Starting Threads
DEBUG:root:Main Creating Work
DEBUG:root:Main Producing Work : 0
DEBUG:root:Main Producing Work : 1
DEBUG:root:Main Producing Work : 2
DEBUG:root:Main Producing Work : 3
DEBUG:root:Main Producing Work : 4
DEBUG:root:  Thread Thread-20 : Getting Value 4
DEBUG:root:  Thread Thread-20 : Getting Value 3
DEBUG:root:  Thread Thread-20 : Getting Value 2
DEBUG:root:  Thread Thread-22 : Getting Value 1
DEBUG:root:  Thread Thread-18 : Getting Value 0
DEBUG:root:Main Producing Work : 5
DEBUG:root:Main Producing Work : 6
DEBUG:root:Main Producing Work : 7
DEBUG:root:Main Producing Work : 8
DEBUG:root:Main Producing Work : 9
DEBUG:root:  Thread Thread-21 : Getting Value 9
DEBUG:root:  Thread Thread-21 : Getting Value 8
DEBUG:root:  Thread Thread-21 : Getting Value 7
DEBUG:root:  Thread Thread-21 : Getting Value 6
DEBUG:root:  Thread Thread-18 : Getting Value 5
DEBUG:root:Main Producing Work : 10
DEBUG:root:Main Producing Work : 11
DEBUG:root:Main Producing Work : 12

# Queue & Workers
* Our main thread creates some work for a pool of workers to complete
* Rather than managing access to the work array, we'll use a Queue
* Queue handles both the Condition and Lock

In [13]:
q = Queue()
results = []

# Creates 25 pieces of work on the queue
def create_work():
    for i in range(25):
        q.put(i)
        
# Works the queue
def do_work():
    while True:
        # Take a number off the queue & multiply by 4
        number = q.get()
        if number is None:
            logger.debug(f'  Thread {threading.current_thread().name} Finished')
            break
            
        logger.debug(f'  Thread {threading.current_thread().name} : Getting Value {number}')
        result = (number, number * 4)
        
        # Append to the results
        time.sleep(1)
        results.append(result)
        
        # Signal done
        q.task_done()
   
# Create daemon five worker threads
logger.debug('Main Starting Threads')
workers = 5
threads = [Thread(target=do_work, daemon=True) for i in range(workers)]
for t in threads: t.start()

# Run the produce function
logger.debug('Main Creating Work')
create_work()

# Join the queue
logger.debug('Main Joining Queue')
q.join()

# Join the threads
logger.debug(results)

# Stop workers
logger.debug('Stopping Workers Threads')
for i in range(workers): q.put(None)
for t in threads: t.join()

DEBUG:root:Main Starting Threads
DEBUG:root:Main Creating Work
DEBUG:root:Main Joining Queue
DEBUG:root:  Thread Thread-23 : Getting Value 0
DEBUG:root:  Thread Thread-25 : Getting Value 1
DEBUG:root:  Thread Thread-26 : Getting Value 2
DEBUG:root:  Thread Thread-27 : Getting Value 3
DEBUG:root:  Thread Thread-24 : Getting Value 4
DEBUG:root:  Thread Thread-23 : Getting Value 5
DEBUG:root:  Thread Thread-25 : Getting Value 6
DEBUG:root:  Thread Thread-26 : Getting Value 7
DEBUG:root:  Thread Thread-27 : Getting Value 8
DEBUG:root:  Thread Thread-24 : Getting Value 9
DEBUG:root:  Thread Thread-23 : Getting Value 10
DEBUG:root:  Thread Thread-25 : Getting Value 11
DEBUG:root:  Thread Thread-26 : Getting Value 12
DEBUG:root:  Thread Thread-27 : Getting Value 13
DEBUG:root:  Thread Thread-24 : Getting Value 14
DEBUG:root:  Thread Thread-23 : Getting Value 15
DEBUG:root:  Thread Thread-25 : Getting Value 16
DEBUG:root:  Thread Thread-26 : Getting Value 18
DEBUG:root:  Thread Thread-27 : Get

# Producer - Consumer
* We extend the pattern
* Now we have a producer that creates a between 1 and 5 jobs
* We have a worker that waits for work
* The worker can do three things before needing a break

## Work Loop

In [14]:
q = Queue()
producing = True

def produce():
    loop = 1
    while producing == True:
        for i in range(random.randint(1,5)):
            work = f'Job {loop}:{i + 1}'
            logger.debug(f'Producing {work}')
            q.put(work)
        loop += 1
        time.sleep(3)

def consume():
    completed = 0
    while True:
        jobs = q.qsize()
        job  = q.get()
        
        if job is None:
            break
        
        logger.debug(f'  Consuming: {job} Remaining: {jobs}')
        q.task_done()
        
        completed += 1
        if completed > 3:
            logger.debug(f'  Consumer: Taking Break')
            time.sleep(2)
            completed = 0

producer = Thread(target=produce)
consumer = Thread(target=consume)

producer.start()
consumer.start()

DEBUG:root:Producing Job 1:1
DEBUG:root:Producing Job 1:2
DEBUG:root:  Consuming: Job 1:1 Remaining: 0
DEBUG:root:Producing Job 1:3
DEBUG:root:Producing Job 1:4


## Cleanup

In [15]:
producing = False
q.put(None)
producer.join()
consumer.join()

DEBUG:root:  Consuming: Job 1:2 Remaining: 1
DEBUG:root:Producing Job 1:5
DEBUG:root:  Consuming: Job 1:3 Remaining: 2
DEBUG:root:  Consuming: Job 1:4 Remaining: 3
DEBUG:root:  Consumer: Taking Break
